In [ ]:
#!pip install ipympl
%load_ext autoreload
%autoreload 2
import fitting as fit
import data_utils as dat
import vis
import plotly.graph_objects as go
import numpy as np

# Modelling time dependence

In [ ]:
# Step 1: fit a power law to loss vs time:

msize, task = "410m", "arxiv"
df_llc, df_loss = dat.load_dfs(msize, data_path="data")
llc, loss, step = dat.trim_trace(df_llc, df_loss, task, 1000, 70000)

loss_result = fit.min_fit(step, loss, fit.ShiftedPowerLaw2, rel_noise=True)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=step, y=loss, mode='markers', 
    marker=dict(color='blue', size=8),
    name='Observed points'))
vis.plot_result(fig, step, loss_result, name="PowerLaw")
fig.update_layout(width=800, height=600, title="Loss vs Time")
fig.update_xaxes(title_text="Step", type="log")
fig.update_yaxes(title_text="Loss")
fig.show()

In [ ]:
# Step 2: fit LLC vs time
# Both models look pretty good

# We have two candidates: log and powerlaw (with some quite different parameters to usual)
result_pwr = fit.min_fit(step, llc, fit.ShiftedPowerLaw, rel_noise=True, par0=[10., -10., 0.001])
result_log = fit.min_fit(step, llc, fit.ShiftedLogarithm, rel_noise=True)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=step, y=llc, mode='markers', 
    marker=dict(color='blue', size=8),
    name='Observed points'))

vis.plot_result(fig, step, result_pwr, name="PowerLaw")
vis.plot_result(fig, step, result_log, name="Logarithm")


fig.update_xaxes(title_text="Step", type="log")

fig.update_yaxes(title_text="LLC")
fig.update_layout(width=800, height=600, title="LLC vs Time")
fig.show()

# What does this say about LLC vs loss?

In [ ]:
# now fit a loss-llc relationship


fig = go.Figure()

fig.add_trace(go.Scatter(
    x=loss, y=llc, mode='markers', 
    marker=dict(color='blue', size=8),
    customdata=step, hovertemplate='Step: %{customdata}<extra></extra>',
    name='Observed points'))

#e_result = fit.min_fit(loss, llc, fit.ShiftedExponential, rel_noise=True)
# p_result = fit.min_fit(loss, llc, fit.ShiftedPowerLaw, rel_noise=True)

# Note - because i'm putting loss on the x axis, only ord_fit can do this

d_result = fit.odr_fit(loss, llc, fit.DoubleShiftedPowerLaw, rel_noise=True)
vis.plot_result(fig, loss, d_result, name="Direct (double shifted)")


p_result = fit.odr_fit(loss, llc, fit.XShiftedPowerLaw2, rel_noise=True)
vis.plot_result(fig, loss, p_result, name="Direct (Power Law)")


#vis.plot_result(fig, loss, e_result, name="Direct (Exponential)")

# Plot the parametric relationship too
q = vis.x_plot(step)  # resample at higher resolution
q_llc = result_pwr.f(q)
q_loss = loss_result.f(q)
fig.add_trace(go.Scatter(
    x=q_loss, y=q_llc, mode='lines', 
    name='Parametric (pwr-pwr)'))

u_llc = result_log.f(q)
fig.add_trace(go.Scatter(
    x=q_loss, y=u_llc, mode='lines', 
    name='Parametric (pwr-log)'))


fig.update_xaxes(title_text="loss")  #, type="log")
fig.update_yaxes(title_text="LLC")
fig.update_layout(width=800, height=600, title=f"LLC vs Loss {task}@{msize}")
fig.show()
